In [ ]:
import numpy as np
import copy

In [ ]:
list(range(2,5))

In [ ]:
def LowerTriangularization(dim):
        for i in range(dim):
            Li = np.zeros((dim, dim))
            for j in range(i+1, dim):
                print(i,j)

LowerTriangularization(3)

In [ ]:
import numpy as np

def lower_triangularization(A):
    dim = A.shape[0]
    L = []
    for i in range(dim):
        for j in range(i + 1, dim):
            Li = np.eye(dim)
            Li[j, i] = A[j, i] / A[i, i]
            A[j, :] = A[j, :] - (A[j, i] / A[i, i]) * A[i, :]
            L.append(Li)
            print(Li)
    
    return L, A

In [152]:
import numpy as np
import copy

class GaussianElimination:
    """
    A class to perform Gaussian Elimination on a given matrix A to decompose it into
    permutation (P), lower triangular (L), diagonal (D), and upper triangular (U) matrices.
    """

    def __init__(self, A):
        """
        Initialize the GaussianElimination class with a matrix A.

        Parameters:
        -----------
        A : np.ndarray
            The matrix to be decomposed.
        """
        self.A = A.astype(np.float64)  # Ensure the matrix A is of dtype np.float64
        self.A_ = copy.deepcopy(self.A)  # Create a deep copy of A to avoid modifying the original matrix
        self.initialize()

    def initialize(self):
        """
        Initialize matrices P, L, D, and U based on the dimension of matrix A.
        """
        self.dim = self.A.shape[0]

        # Permutation matrix initialized as an identity matrix with dtype np.float64
        self.P = [np.eye(self.dim, dtype=np.float64)]
        
        # Initialize L as the identity matrix and store intermediate operations
        self.L_operations = []
        self.L = np.eye(self.dim)
        
        # Initialize diagonal matrix D and upper triangular matrix U with zeros
        self.D = np.zeros((self.dim, self.dim), dtype=np.float64)
        self.U = np.zeros((self.dim, self.dim), dtype=np.float64)

    def first_non_zero_index(self, idx):
        """
        Find the index of the first non-zero element in the sub-array starting from index `idx`.
        
        Parameters:
        -----------
        idx : int
            The starting index for the search in the sub-array.
        
        Returns:
        --------
        int
            The index of the first non-zero element in the sub-array.
        """
        column = self.A_[idx:, idx]
        non_zero_indices = np.nonzero(column)[0]

        return non_zero_indices[0] + idx

    def permutation_matrix(self, col1, col2):
        """
        Swap two columns in the permutation matrix P.

        Parameters:
        -----------
        col1 : int
            The first column to swap.
        col2 : int
            The second column to swap.
        """
        Pi = np.eye(self.dim)
        Pi[:, [col1, col2]] = Pi[:, [col2, col1]]
        self.P = [Pi] + self.P
        # print("Pi: ", Pi)
        return Pi

    def swap_rows_A_(self, row1, row2):
        """
        Swap two rows in the matrix A_.

        Parameters:
        -----------
        row1 : int
            The first row to swap.
        row2 : int
            The second row to swap.

        Returns:
        --------
        np.ndarray
            The matrix A_ with the specified rows swapped.
        """
        self.A_[[row1, row2], :] = self.A_[[row2, row1], :]
        return self.A_

    def lower_triangularization(self):
        """
        Perform the lower triangularization of matrix A.

        This method transforms matrix A into a lower triangular form by applying
        a series of row operations and stores the intermediate lower triangular
        matrices in L_operations.
        """
        for i in range(self.dim):
            if self.A_[i, i] == 0:
                interchange_idx = self.first_non_zero_index(i)
                Pi = self.permutation_matrix(i, interchange_idx)  # Swap columns in permutation matrix
                self.swap_rows_A_(i, interchange_idx)  # Swap rows in A_
                # Store operations for P and L matrices
                self.L_operations.append(Pi)
            else:
                Pi = np.eye(self.dim)
            
            Li = np.eye(self.dim, dtype=np.float64)  # Identity matrix to accumulate row operations
            
            for j in range(i + 1, self.dim):
                # Compute the multiplier for row transformation
                Li[j, i] = self.A_[j, i] / self.A_[i, i]
                # Update row j of A_ by subtracting the scaled row i
                self.A_[j, :] -= (self.A_[j, i] / self.A_[i, i]) * self.A_[i, :]
            self.L_operations.append(Li)  # Store the lower triangular matrix

        # print(self.L.shape)
        # Combine all lower triangular matrices to form the final L matrix
        for li in self.L_operations:
            # print(li)
            self.L = np.matmul(self.L, li)
            # print(self.L.shape)
        for pi in self.P:
            self.L = np.matmul(pi, self.L)
        
    def extract_diagonal_and_upper(self):
        """
        Extract the diagonal (D) and upper triangular (U) matrices from A_.
        """
        for i in range(self.dim):
            self.D[i, i] = self.A_[i, i]  # Extract diagonal elements
            self.U[i, :] = self.A_[i, :]  # Extract upper triangular part

        # Normalize U to ensure it has ones on the diagonal
        for i in range(self.dim):
            if self.D[i, i] != 0:
                self.U[i, :] /= self.D[i, i]

    def decompose(self):
        """
        Perform Gaussian Elimination to decompose matrix A into L, D, and U.

        Returns:
        --------
        tuple of np.ndarray
            A tuple containing the lower triangular matrix (L),
            diagonal matrix (D), and upper triangular matrix (U).
        """
        self.lower_triangularization()
        self.extract_diagonal_and_upper()
        return self.L, self.D, self.U

In [153]:
A = np.array([[1,3,2], [-2,-6,1], [2,5,7]], dtype=np.float64)

GE = GaussianElimination(A)
L,D,U = GE.decompose()

In [154]:
L

array([[ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [-2.,  0.,  1.]])

In [149]:
D

array([[ 1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  5.]])

In [150]:
U

array([[ 1.,  3.,  2.],
       [-0.,  1., -3.],
       [ 0.,  0.,  1.]])

In [151]:
np.matmul(np.matmul(L,D), U)

array([[ 1.,  3.,  2.],
       [ 2.,  6.,  9.],
       [-2., -7., -1.]])

In [122]:
np.matmul(D,U)

array([[ 1.,  3.,  2.],
       [ 0., -1.,  3.],
       [ 0.,  0.,  5.]])